In [ ]:
%matplotlib inline

# Create a surrogate discipline.

We want to build a [Discipline][gemseo.core.discipline.discipline.Discipline]
based on a regression model approximating the following discipline
with two inputs and two outputs:

- $y_1=1+2x_1+3x_2$
- $y_2=-1-2x_1-3x_2$

over the unit hypercube $[0,1]\times[0,1]$.
For that,
we use a [SurrogateDiscipline][gemseo.disciplines.surrogate.SurrogateDiscipline]
relying on a
[BaseRegressor][gemseo.mlearning.regression.algos.base_regressor.BaseRegressor].


In [ ]:
from __future__ import annotations

from numpy import array

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import create_surrogate
from gemseo import sample_disciplines

## Create the discipline to learn

We can implement this analytic discipline by means of the
[AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline] class.



In [ ]:
expressions = {"y_1": "1+2*x_1+3*x_2", "y_2": "-1-2*x_1-3*x_2"}
discipline = create_discipline(
    "AnalyticDiscipline", name="func", expressions=expressions
)

## Create the input sampling space

We create the input sampling space by adding the variables one by one.



In [ ]:
design_space = create_design_space()
design_space.add_variable("x_1", lower_bound=0.0, upper_bound=1.0)
design_space.add_variable("x_2", lower_bound=0.0, upper_bound=1.0)

## Create the training dataset

We can build a training dataset
by sampling the discipline using the [sample_disciplines()][gemseo.sample_disciplines]
function with a full factorial design of experiments.



In [ ]:
dataset = sample_disciplines(
    [discipline], design_space, ["y_1", "y_2"], algo_name="PYDOE_FULLFACT", n_samples=9
)

## Create the surrogate discipline

Then, we build the Gaussian process regression model from the dataset and
display this model.



In [ ]:
model = create_surrogate("GaussianProcessRegressor", data=dataset)

## Predict output

Once it is built, we can use it for prediction, either with default inputs



In [ ]:
model.execute()

or with user-defined ones.



In [ ]:
model.execute({"x_1": array([1.0]), "x_2": array([2.0])})